<a href="https://colab.research.google.com/github/omkarjaliparthi/My-Projects/blob/main/_Fashion_MNIST_with_PyTorch_extended_Part_E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt


In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
testset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=False)


100%|██████████| 26.4M/26.4M [00:02<00:00, 13.2MB/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 209kB/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:01<00:00, 3.42MB/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 12.0MB/s]


Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [4]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.softmax(x)
        return x


In [5]:
model = SimpleNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [8]:
epochs = 10
for epoch in range(epochs):
    running_loss = 0.0
    for images, labels in trainloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}")


Epoch 1, Loss: 0.16082039374365673
Epoch 2, Loss: 0.15883736529433207
Epoch 3, Loss: 0.15239834629777652
Epoch 4, Loss: 0.14769468872921107
Epoch 5, Loss: 0.14669081786953247
Epoch 6, Loss: 0.13725448966121623
Epoch 7, Loss: 0.13584812013094805
Epoch 8, Loss: 0.13019111032075464
Epoch 9, Loss: 0.12768483601375494
Epoch 10, Loss: 0.12304047570982848


In [9]:
correct = 0
total = 0
with torch.no_grad():
    for images, labels in testloader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy: {100 * correct / total}%")


Accuracy: 87.79%


In [10]:
class ModifiedNN(nn.Module):
    def __init__(self):
        super(ModifiedNN, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.flatten(x)
        x = torch.nn.functional.gelu(self.fc1(x))
        x = torch.nn.functional.max_pool1d(x.unsqueeze(2), kernel_size=2).squeeze(2)
        x = torch.nn.functional.gelu(self.fc2(x))
        x = torch.nn.functional.max_pool1d(x.unsqueeze(2), kernel_size=2).squeeze(2)
        x = self.fc3(x)
        x = self.softmax(x)
        return x


In [11]:
criterion = nn.CrossEntropyLoss()  # Same as before
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer


In [12]:
epochs = 10
for epoch in range(epochs):
    running_loss = 0.0
    for images, labels in trainloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}")


Epoch 1, Loss: 0.12219159669979517
Epoch 2, Loss: 0.11439466769638251
Epoch 3, Loss: 0.11302237314428253
Epoch 4, Loss: 0.11508669732198087
Epoch 5, Loss: 0.10818532290560667
Epoch 6, Loss: 0.10508317092179394
Epoch 7, Loss: 0.10224944976732325
Epoch 8, Loss: 0.10325195605673992
Epoch 9, Loss: 0.09509878338419839
Epoch 10, Loss: 0.09411780986024428


In [13]:
correct = 0
total = 0
with torch.no_grad():
    for images, labels in testloader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy: {100 * correct / total}%")


Accuracy: 87.62%


In [24]:
optimizer = optim.Adam(model.parameters(), lr=0.0005)


In [25]:
epochs = 3
for epoch in range(epochs):
    running_loss = 0.0
    for images, labels in trainloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}")


Epoch 1, Loss: 0.06364560202022256
Epoch 2, Loss: 0.05743262251523305
Epoch 3, Loss: 0.05521736115039483


In [26]:
correct = 0
total = 0
with torch.no_grad():
    for images, labels in testloader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy: {100 * correct / total}%")


Accuracy: 89.06%


In [34]:
def calculate_accuracy(model, testloader):
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in testloader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total


In [35]:
accuracy_original = calculate_accuracy(original_model, testloader)
accuracy_modified = calculate_accuracy(modified_model, testloader)
print(f"Accuracy of original model: {accuracy_original}%")
print(f"Accuracy of modified model: {accuracy_modified}%")

Accuracy of original model: 10.35%
Accuracy of modified model: 12.02%
